In [2]:
print("iuhgfc")

iuhgfc


In [3]:
%pwd

'd:\\Gai-spots\\Source-Code-Analysis\\research'

In [4]:
!mkdir test_repo

In [5]:
from git import Repo

repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/aTul-07kn/Medi-Chatbot-Medicrolina", to_path=repo_path)

In [6]:
%pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [8]:
docs=loader.load()

In [9]:
docs

[Document(metadata={'source': 'test_repo\\ test.py', 'language': <Language.PYTHON: 'python'>}, page_content=''),
 Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, request \nfrom src.helper import *\nfrom src.prompt import system_prompt\nfrom langchain_pinecone import PineconeVectorStore\nimport os\nfrom dotenv import load_dotenv \nfrom langchain_nvidia_ai_endpoints import ChatNVIDIA\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_nvidia_ai_endpoints import ChatNVIDIA\n\nload_dotenv()\n\napp = Flask(__name__)\n\nPINECONE_API_KEY=os.getenv("PINECONE_API_KEY")\nNVIDIA_API_KEY=os.getenv("NVIDIA_API_KEY")\nos.environ["NVIDIA_API_KEY"]=NVIDIA_API_KEY\n\nembeddings=download_hugging_face_embeddings()\nindex_name="medi-chatbot-index"\n\ndocsearch 

In [10]:
len(docs)

8

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter=RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=500,
    chunk_overlap=50
)

In [12]:
texts=splitter.split_documents(docs)

In [13]:
texts

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, request \nfrom src.helper import *\nfrom src.prompt import system_prompt\nfrom langchain_pinecone import PineconeVectorStore\nimport os\nfrom dotenv import load_dotenv \nfrom langchain_nvidia_ai_endpoints import ChatNVIDIA\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_nvidia_ai_endpoints import ChatNVIDIA'),
 Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='load_dotenv()\n\napp = Flask(__name__)\n\nPINECONE_API_KEY=os.getenv("PINECONE_API_KEY")\nNVIDIA_API_KEY=os.getenv("NVIDIA_API_KEY")\nos.environ["NVIDIA_API_KEY"]=NVIDIA_API_KEY\n\nembeddings=download_hugging_face_embeddings()\nindex_name="medi-chatbot-index"\n\ndocsearch = Pine

In [14]:
len(texts)

15

In [15]:
%pip install -U --quiet langchain-nvidia-ai-endpoints

Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
NVIDIA_API_KEY=os.getenv("NVIDIA_API_KEY")

In [17]:
os.environ["NVIDIA_API_KEY"]=NVIDIA_API_KEY

In [18]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(
  model="NV-Embed-QA",  
  truncate="NONE", 
  )

In [19]:
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(texts, embedding=embedder, persist_directory='./db')

In [20]:
vectordb.persist()

C:\Users\ATUL\AppData\Local\Temp\ipykernel_19988\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [21]:
embd=embedder.embed_query("hello")

In [22]:
len(embd)

1024

In [23]:
llm = ChatNVIDIA(
  model="nvidia/llama-3.1-nemotron-70b-instruct",
  temperature=0.6,
  top_p=1,
  max_tokens=1024,
  stream=True
)

In [24]:
llm

ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='nvidia/llama-3.1-nemotron-70b-instruct', temperature=0.6, top_p=1.0)

In [25]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

C:\Users\ATUL\AppData\Local\Temp\ipykernel_19988\2984388413.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)


In [26]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

In [27]:
question = "what is download_hugging_face_embeddings funtion?"

result = qa(question)
print(result['answer'])

C:\Users\ATUL\AppData\Local\Temp\ipykernel_19988\2264811874.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(question)


Based on the provided context, here's what we can infer about the `download_hugging_face_embeddings` function:

**Function Signature:**
```python
def download_hugging_face_embeddings():
```
**Return Value:**
The function returns an object named `embeddings`.

**Implementation Details:**
Inside the function, it creates an instance of `HuggingFaceEmbeddings` from the `langchain_huggingface.embeddings` module, specifying a particular `model_name`:
```python
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
```
**Inference:**
Given this information, we can infer that:

1. **Purpose:** The primary purpose of `download_hugging_face_embeddings` is to fetch and return a specific Hugging Face embedding model.
2. **Embedding Model:** The function downloads and returns the **`all-MiniLM-L6-v2`** model from the **`sentence-transformers`** repository on Hugging Face.
3. **Return Type:** Although the exact type is not explicitly stated, the return value (`embedd

In [28]:
result = qa("what does the **system_prompt** say")
print(result['answer'])

Based on the provided context, here is a concise answer to the standalone question:

The **system_prompt** says:
* You are an assistant for question-answering tasks.
* Use the following pieces of retrieved context to answer the question.
* If you don't know the answer, say that you don't know.
* Use three sentences maximum and keep the answer concise.


In [29]:
result = qa("Can we improve this prompt, improve it.")
print(result['answer'])

Based on the provided **system_prompt**, here's a concise answer with suggestions for improvement:

**Improvement Suggestions for **system_prompt****

1. **Specify Context Scope**: Clarify what is meant by "the following pieces of retrieved context" (e.g., are they always from a specific source, format, or domain?).
2. **Define Conciseness Boundaries**: Elaborate on "keep the answer concise" by providing a character/token limit or a specific response format (e.g., bullet points, short paragraphs).
3. **Consider Adding a Tone or Style Guideline**: To ensure consistency, suggest a tone (e.g., formal, friendly) or style (e.g., objective, persuasive) for the responses.

**Revised System Prompt Example (incorporating suggestions):**
```
"You are an assistant for question-answering tasks. 
Please respond using the provided context below, 
if insufficient, say 'I don't know'. 
Keep answers within 250 characters, 
using short paragraphs or bullet points, 
and maintain a neutral, informative to